<h1><center>Segmenting and Clustering Neighborhoods in Toronto</center></h1>
<h2><center>Suraj Eswaran</center></h2>

In [66]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://www.seetorontonow.com/wp-content/uploads/2021/01/header-toronto-skyline-sunset-CityofToronto-1920x810-1.jpg")

Toronto is the capital city of the Canadian province of Ontario. With a recorded population of 2,731,571,it is the most populous city in Canada and the fourth most populous city in North America. The city is the anchor of the Golden Horseshoe, an urban agglomeration of 9,245,438 people (as of 2016) surrounding the western end of Lake Ontario,while the Greater Toronto Area (GTA) proper had a 2016 population of 6,417,516. Toronto is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world.

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in Toronto City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the _k_-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in Toronto City and their emerging clusters.

## 1. Import the Libraries

In [7]:
import pandas as pd 
print(pd.__version__)
!pip install pandas==1.0.3
print(pd.__version__)

1.0.5
1.0.5


In [8]:
import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [9]:
!pip install folium

     |████████████████████████████████| 94 kB 7.5 MB/s  eta 0:00:01


In [10]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [11]:
from bs4 import BeautifulSoup
import lxml
from IPython.display import display_html

## 2. Download and Explore Dataset

In [12]:
response = requests.get(url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969", )
soup = BeautifulSoup(response.content, 'lxml')
print(soup.title)
canada_table=str(soup.table)
display_html(canada_table,raw=True)

<title>List of postal codes of Canada: M - Wikipedia</title>


Postal Code,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M8A,Not assigned,Not assigned
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"


## 3. Transform the data into a pandas dataframe

In [13]:
dfs = pd.read_html(canada_table)
df=dfs[0]
# Dropping the rows where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']

# Combining the neighbourhoods with same Postalcode
df2 = df1.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])
df2

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [15]:
#df2.shape
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()
df3=pd.merge(df2,lat_lon)
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [16]:
df4= df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


## 4. Use geopy library to get the latitude and longitude values of Toronto City.

In [18]:
address='Toronto,ON,Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## 5. Create a map of Toronto with neighborhoods superimposed on top.

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'],df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 6. Define Foursquare Credentials and Version

In [20]:
CLIENT_ID = 'RCI2LJCNCNGYLRIOPU2MSCHC3ZGBSKVPLF5FRG12DA1LNIGL' # your Foursquare ID
CLIENT_SECRET = 'D254LXL03DB0ZGUI3ADD32ELMIX4DBMVYX5Q2V1DH2LCM1Y4' # your Foursquare Secret
ACCESS_TOKEN = 'UXC23PT1T1VQQMHIBYNMKY2YYFO3WJEMCWM1HQ3P5MPK4S3S' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RCI2LJCNCNGYLRIOPU2MSCHC3ZGBSKVPLF5FRG12DA1LNIGL
CLIENT_SECRET:D254LXL03DB0ZGUI3ADD32ELMIX4DBMVYX5Q2V1DH2LCM1Y4


## 7. Let's explore the first neighborhood in our dataframe

In [21]:
df4['Neighbourhood'].iloc[0]

'Regent Park, Harbourfront'

## 8. Get the neighborhood's latitude and longitude values

In [22]:
nei_lat=df4['Latitude'].iloc[23]
nei_lon=df4['Longitude'].iloc[23]
nei_name=df4['Neighbourhood'].iloc[23]

## 9. Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [23]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, nei_lat, nei_lon, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=RCI2LJCNCNGYLRIOPU2MSCHC3ZGBSKVPLF5FRG12DA1LNIGL&client_secret=D254LXL03DB0ZGUI3ADD32ELMIX4DBMVYX5Q2V1DH2LCM1Y4&ll=43.6616083,-79.46476329999999&v=20180604&radius=500&limit=100'

In [24]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6051735a62d9b434d7f02e71'},
 'response': {'headerLocation': 'High Park',
  'headerFullLocation': 'High Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 43.6661083045, 'lng': -79.458554540355},
   'sw': {'lat': 43.657108295499995, 'lng': -79.47097205964498}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd30e129854d13acdbefc4d',
       'name': 'Lithuania Park',
       'location': {'address': '155 Oakmount Rd',
        'crossStreet': 'at Glenlake & Keele',
        'lat': 43.658666653470426,
        'lng': -79.46303844451904,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.658666653470426,
          'lng': -79.46303844451904}],
        'distan

## 10. Let's borrow the get_category_type function from the Foursquare lab.

In [25]:
# keep only columns that include venue name, and anything that is associated with location
#filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
#dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
#dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
#dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered

## 11. Clean the json and structure it into a pandas dataframe

In [26]:
#assign relevant part of JSON to venues
venues=results['response']['groups'][0]['items']
#venues = results['response']['groups']

# tranforms venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

filtered_columns = ['venue.name', 'venue.categories','venue.location.lat','venue.location.lng'] 
dataframe_filtered = dataframe.loc[:, filtered_columns]
#dataframe_filtered

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Lithuania Park,Park,43.658667,-79.463038
1,nodo,Italian Restaurant,43.665303,-79.465621
2,Indie Alehouse,Gastropub,43.665475,-79.465290
3,Hole in the Wall,Bar,43.665296,-79.465118
4,Junction Flea,Flea Market,43.665258,-79.462868


#  12. Explore Neighborhoods in Regent Park


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## 13. Run the above function on each neighborhood and create a new dataframe called venues_toronto

In [28]:
venues_toronto = getNearbyVenues(names=df4['Neighbourhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harb

## 14. Number of venues returned for each neighborhood

In [31]:
venues_toronto.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


## 15. Analyze Each Neighborhood

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_toronto[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = venues_toronto['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.016667,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.016667,0.050000,0.000000,0.000000,0.0,0.016667,0.016667,0.0,0.033333,0.0,0.0,0.016667,0.0,0.000000,0.0,0.000000,0.0,0.016667,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.016667,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.016667,0.05,0.083333,0.0,0.0,0.0,0.0,0.0,0.016667,0.000000,0.016667,0.000000,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.016667,0.000000,0.0,0.0,0.0,0.000000,0.0,0.016667,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0000,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.0,0.016667,0.016667,0.0,0.0,0.0,0.000000,0

## 16. Let's print each neighborhood along with the top 5 most common venues

In [68]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1        Bakery  0.05
2  Cocktail Bar  0.05
3      Pharmacy  0.03
4      Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2     Coffee Shop  0.08
3             Bar  0.04
4   Burrito Place  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Gym / Fitness Center  0.06
1               Brewery  0.06
2            Skate Park  0.06
3            Restaurant  0.06
4      Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2          Boutique  0.07
3           Airport  0.07
4             Plane  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Resta

## 17. Let's put that into a pandas dataframe

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## 18. create the new dataframe and display the top 10 venues for each neighborhood.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Cheese Shop,Pharmacy,Seafood Restaurant,Beer Bar,Farmers Market,Japanese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Grocery Store,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Garden,Skate Park,Light Rail Station,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Recording Studio,Auto Workshop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Coffee Shop,Plane,Harbor / Marina,Boat or Ferry,Airport Lounge
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Thai Restaurant,Salad Place,Restaurant


## 19. Cluster Neighborhoods

In [71]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 1, 0,
       0, 0, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [61]:
toronto_merged = df4
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Yoga Studio,Dessert Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Distribution Center,Sandwich Place,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Hotel,Pizza Place
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Gastropub,Cosmetics Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Farmers Market,Hotel,Italian Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Pub,Trail,Health Food Store,Neighborhood,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Cheese Shop,Pharmacy,Seafood Restaurant,Beer Bar,Farmers Market,Japanese Restaurant
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Thai Restaurant,Salad Place,Restaurant
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Candy Store
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Clothing Store,Restaurant,Thai Restaurant,Hotel,Deli / Bodega,Bakery,Gym,Lounge
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Bakery,Pharmacy,Smoke Shop,Music Venue,Supermarket,Middle Eastern Restaurant,Café,Pool,Bar,Bank


In [63]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 20. Examine Clusters

#### Cluster 1 

In [73]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Yoga Studio,Dessert Shop
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Distribution Center,Sandwich Place,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Hotel,Pizza Place
15,Downtown Toronto,0,Coffee Shop,Café,Gastropub,Cosmetics Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Farmers Market,Hotel,Italian Restaurant
19,East Toronto,0,Asian Restaurant,Pub,Trail,Health Food Store,Neighborhood,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run
20,Downtown Toronto,0,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Cheese Shop,Pharmacy,Seafood Restaurant,Beer Bar,Farmers Market,Japanese Restaurant
24,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Thai Restaurant,Salad Place,Restaurant
25,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Candy Store
30,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Restaurant,Thai Restaurant,Hotel,Deli / Bodega,Bakery,Gym,Lounge
31,West Toronto,0,Bakery,Pharmacy,Smoke Shop,Music Venue,Supermarket,Middle Eastern Restaurant,Café,Pool,Bar,Bank


#### Cluster 2

In [74]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Gym,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 3

In [75]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
91,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 4

In [76]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,3,Dim Sum Restaurant,Park,Bus Line,Swim School,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 5

In [77]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Toronto/York,4,Convenience Store,Breakfast Spot,Brewery,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
